In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [3]:
train_tape = "Z:/rppg/pure_datatape_450x32x32_train.h5"
valid_tape = "Z:/rppg/pure_datatape_450x32x32_valid.h5"

#train = to_tf(train).cache(f'{tmp}/ours_32_pure')
train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)
train, valid = to_tf(train).cache(), to_tf(valid).cache()

In [2]:
m4 = M_1()
m4.compile(optimizer='Nadam', loss='mae')
m4.build(input_shape=(None, 450, 32, 32, 3))
print(f'Flops per frame:{get_flops(m4, input_sig=[tf.TensorSpec([1, 450, 32, 32, 3])])/450:.0f}')
m4.summary()

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:633997
Model: "m_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 450, 64)           196672    
_________________________________________________________________
spectral_transform (Spectral multiple                  82560     
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  41024     
_________________________________________________________________
sequential_2 (Sequential)    (None, 450, 64)           256       
_________________________________________________________________
spectral_transform_1 (Spectr multiple                  49792     
_________________________________________________________________
conv1d_4 (Conv1D)            multiple                  10272     
_________________

In [ ]:
m4.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/m4_pure.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
m4.load_weights('../weights/m4_pure.h5')

In [5]:
m4.load_weights('../weights/m4_pure.h5')

In [8]:
eval_on_dataset(test_set_CCNU, m4, 450, (32, 32), step=1, batch=32, save='../results/ours_32_PURE_CCNU.h5')
get_metrics('../results/ours_32_PURE_CCNU.h5', use_filter=False)

100%|██████████| 179/179 [11:09<00:00,  3.74s/it]


{'Sliding window': {'MAE': 4.105, 'RMSE': 10.066, 'R': 0.59617},
 'Whole video': {'MAE': 2.263, 'RMSE': 7.058, 'R': 0.75107}}

In [7]:
eval_on_dataset(test_set_CCNU_rPPG, m4, 450, (32, 32), step=1, batch=1, save='../results/ours_32_PURE_CCNU_rPPG.h5')
get_metrics('../results/ours_32_PURE_CCNU_rPPG.h5', use_filter=False)

100%|██████████| 54/54 [02:25<00:00,  2.69s/it]


{'Sliding window': {'MAE': 3.688, 'RMSE': 9.401, 'R': 0.65644},
 'Whole video': {'MAE': 1.866, 'RMSE': 7.168, 'R': 0.79342}}

In [6]:
eval_on_dataset(test_set_UBFC_rPPG2, m4, 450, (32, 32), step=1, batch=1, save='../results/ours_32_PURE_UBFC.h5')
get_metrics('../results/ours_32_PURE_UBFC.h5', use_filter=False)

100%|██████████| 42/42 [01:01<00:00,  1.47s/it]


{'Sliding window': {'MAE': 1.68, 'RMSE': 4.023, 'R': 0.97434},
 'Whole video': {'MAE': 1.344, 'RMSE': 1.967, 'R': 0.99464}}